In [1]:
from dotenv import load_dotenv
import os

ENV_PATH = os.path.join("/home/lordzuko/work/speech-editor/.env")
load_dotenv(dotenv_path=ENV_PATH)

MODE = os.environ["MODE"]
DEBUG = os.environ["DEBUG"] == "True"
# DEBUG=True
DB = os.environ["DB"]
DB_HOST = os.environ["DB_HOST"]
USERNAME = "" if os.environ["USERNAME"] == "" else os.environ["USERNAME"]
PASSWORD = "" if os.environ["PASSWORD"] == "" else os.environ["PASSWORD"]


In [2]:
import certifi
from mongoengine import connect
from pymongo import ReadPreference

def db_init():
    return connect(
        host=f"mongodb+srv://{DB_HOST}/{DB}?retryWrites=true&w=majority&ssl=true",
        # host=f"mongodb://{APP_HOST}/{APP_DB}",
        username=USERNAME,
        password=PASSWORD,
        authentication_source="admin",
        read_preference=ReadPreference.PRIMARY_PREFERRED,
        # maxpoolsize=MONGODB_POOL_SIZE,
        tlsCAFile=certifi.where(),
    )


In [3]:
db_init()

MongoClient(host=['ac-mwbscfs-shard-00-00.fnblyuu.mongodb.net:27017', 'ac-mwbscfs-shard-00-02.fnblyuu.mongodb.net:27017', 'ac-mwbscfs-shard-00-01.fnblyuu.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', ssl=True, authsource='admin', replicaset='atlas-46h5qd-shard-0', read_preference=PrimaryPreferred(tag_sets=None, max_staleness=-1, hedge=None), ssl_ca_certs='/home/lordzuko/miniconda3/envs/se_daft/lib/python3.8/site-packages/certifi/cacert.pem')

In [4]:
import datetime

from mongoengine import (BooleanField, DateTimeField, DictField, Document,
                         EmailField, ListField, StringField, IntField)


class Text(Document): # pylint-disable: too-few-public-methods
    """
    Docoment class for the text data
    """
    meta = {"collection": "text"}
    wav_name = StringField(required=True)
    ref_style = StringField(required=True)
    text = StringField(required=True)
    # utt_len = IntField(required=True)


In [6]:
import yaml
config = yaml.load(
    open("/home/lordzuko/work/speech-editor/conf/config.yaml", "r"), Loader=yaml.FullLoader
)


In [8]:
counter = 1
max = str(20)
for st, st_path in config["ref_style"].items():
    for t_k, t_v in config["text"].items():
        data = {}
        data["wav_name"] = f"{counter}/{max}.wav"
        data["ref_style"] = st
        data["text"] = t_v
        success = Text(**data).save()
        counter+=1